In [ ]:
from matplotlib import pyplot as plt

import os, yaml, random, natsort
import numpy as np
import cv2

from functools import reduce
from itertools import compress

# Camera

The relationship between the 3D point **'X'** and its image projection **'x'** is given by:


\begin{align}
sx = K\begin{vmatrix} R & t\end{vmatrix}X = PX
\end{align}

\begin{align}
P = K\begin{vmatrix} R & t\end{vmatrix}
\end{align}


where **'s'** is an arbitrary scale factor, (**'R'**, **'t'**), called the **extrinsic parameters** is the rotation and translation which relates the world coordinate system to camera coordinate system and **'K'** is the camera **instrinsic matrix**. The 3x4 matrix **'P'** is called the camera **projection matrix**.

In [ ]:
class Camera(object):
    def __init__(self, camera_id:str):
        super().__init__()
        self.camera_id = camera_id
        
        self.K = np.eye(3, dtype=np.float64).reshape((3,3))
        self.R = np.eye(3, dtype=np.float64).reshape((3,3))
        self.T = np.zeros(3, dtype=np.float64).reshape((3,1))
        self.P = np.dot(self.getIntrisicMatrix(), self.getExtrinsicMatrix()).reshape((3,4))
        self.dist_coeff = np.zeros((5,1), dtype=np.float64)
    
    def getIntrisicMatrix(self):
        return self.K
    
    def getIntrisicParameters(self):
        return self.K, self.dist_coeff
    
    def getExtrinsicMatrix(self):
        return np.concatenate((self.R, self.T), axis=1).reshape((3,4))
    
    def getExtrinsicParameters(self):
        return self.R, self.T
    
    def getProjectionMatrix(self):
        return self.P
    
    def saveCameraConfig(self, save_path):
        data = {'K':          np.asarray(self.K).tolist(), 
                'dist_coeff': np.asarray(self.dist_coeff).tolist()}
        try:
            with open(os.path.join(save_path, self.camera_id+".yaml"), "w") as f:
                yaml.dump(data, f)
        except FileNotFoundError:
            print('Path does not exist')
            
    def loadCameraConfig(self, config_path):
        try:
            with open(config_path) as f:
                loaded_dict = yaml.load(f, Loader=yaml.FullLoader)
                
                self.K = np.asarray(loaded_dict.get('K')).reshape((3, 3))
                self.dist_coeff = np.asarray(loaded_dict.get('dist_coeff')).reshape((5, 1))
        except FileNotFoundError:
            print('File does not exist')

# Simple Calibration (Intrinsic parameters)

The task of camera calibration is to determine the parameters of thetransformation between an object in 3D space and the 2D image observed by the camera from visual information (images). The transformation includes:

- **Extrinsic parameters**: **orientation** (rotation) and **location** (translation) of the camera, i.e., (**R, t**);

- **Intrinsic parameters** (**K**):  characteristics of the camera where

\begin{align}
K = \begin{vmatrix} 
        \alpha & \rho & u_0 \\
        0 & \beta & v_0 \\
        0 & 0 & 1 
    \end{vmatrix}
\end{align}

The **rotation matrix**, although consisting of 9 elements, only **has 3 degrees of freedom**. The **translation vector has 3 parameters**. Therefore, there are 6 extrinsic parameters and 5 intrinsic parameters, leading to in total 11 parameters

In [ ]:
from IPython.display import clear_output
import time

class CameraCalibrator(object):
    def __init__(self, camera:Camera, board_size=(9,6), flags=cv2.CALIB_CB_FAST_CHECK):
        self.board_size = board_size
        self.flags = flags
        self.calibrated = False
        self.img_size = (0, 0)
        
        # Arrays to store object points and image points from all the images.
        self.filenames = []       # Name of the images that have been used for calibration
        self.find_corners = []    # Index of images that worked, finding corners
        self.objpoints = []       # 3d point in real world space
        self.imgpoints = []       # 2d points in image plane.
        
        # Camera to calibrate, there you will find the parameters to calibrate 
        # (intrinsic matrix [K] and distortion coefficient)
        self.camera = camera
    
    def findChessboardCorners(self, img_dir, show=False):
        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        board_elements = reduce(lambda x, y: x*y, self.board_size)
        objp = np.zeros((board_elements,3), np.float32)
        objp[:,:2] = np.mgrid[0:self.board_size[0],0:self.board_size[1]].T.reshape(-1,2)
        
        # Arrays to store object points and image points from all the images.
        self.filenames = []       # Name of the images that have been used for calibration
        self.find_corners = []    # Index of images that worked, finding corners
        self.objpoints = []       # 3d point in real world space
        self.imgpoints = []       # 2d points in image plane.
        
        # termination criteria (subpixel process)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        img_files = natsort.natsorted(os.listdir(img_dir))
        for img_file in img_files:
            self.filenames.append(os.path.join(img_dir, img_file))
            img = cv2.imread(os.path.join(img_dir, img_file))
            if len(img.shape) == 3 and img.shape[2] == 3:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray = img
            
            ret, corners = cv2.findChessboardCorners(gray, self.board_size, flags = cv2.CALIB_CB_ADAPTIVE_THRESH |
                    cv2.CALIB_CB_NORMALIZE_IMAGE | self.flags)
            
            # If found image points, refine them
            if isinstance(corners, np.ndarray):
                corners = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
                        
            self.find_corners.append(ret)
            self.objpoints.append(objp)
            self.imgpoints.append(corners)
     
            if ret and show:    
                # Draw and display the corners
                cv2.drawChessboardCorners(img, self.board_size, corners, ret)
                fig = plt.figure(figsize=(10, 10))
                plt.imshow(img)
                plt.title('Calibration')
                plt.xticks([])
                plt.yticks([]) 
                plt.show()    
                time.sleep(0.4)
                clear_output(wait=True)
        
        self.img_size = gray.shape[::-1]
        
    
    def calibrate(self, img_dir, show_calibration=False):
        self.findChessboardCorners(img_dir, show_calibration)
        
        cv2.calibrateCamera(
            self.getObjPoints(), self.getImgPoints(),
            self.img_size, self.camera.K,
            self.camera.dist_coeff,
            flags = self.flags
        )
        
        self.calibrated = True
            
    def getIntrisicCameraParameters(self):
        return self.camera.getIntrisicParameters()
    
    def getExtrinsicCameraParameters(self):
        return self.camera.getExtrinsicParameters()
        #return self.R, self.P
    
    def mapCameraCoordinateToPixelCoordinate(self, pixel_scaling_factor):
        if not self.calibrated:
            return
            
        new_intrinsics, roi = cv2.getOptimalNewCameraMatrix(self.camera.K. self.camera.dist_coeff,
                                                            self.image_size, pixel_scaling_factor)
        return new_intrinsics, roi
    
    def getPoints(self, ignore_none=True):
        ''' 
            return object points (points in world coordinates) and the image points
            (points in images coordinates).
            
            @params ignore_none: Used to ignore None points generated because in this image the algorithm
                was no able to find the corners.
        '''
        return self.getObjPoints(ignore_none), self.getImgPoints(ignore_none)
    
    def getImgPoints(self, ignore_none=True):
        ''' 
            return image points(points in images coordinates)
            
            @params ignore_none: Used to ignore None points generated because in this image the algorithm
                was no able to find the corners.
        '''
        if ignore_none:
            return list(compress(self.imgpoints, self.find_corners))
        else:
            return self.imgpoints
    
    def getObjPoints(self, ignore_none=True):
        ''' 
            return object points (points in world coordinates)
            
            @params ignore_none: Used to ignore None points generated because in this image the algorithm
                was no able to find the corners.
        '''
        if ignore_none:
            return list(compress(self.objpoints, self.find_corners))
        else:
            return self.objpoints
        
    def getImagesFilenames(self, ignore_none=True):
        if ignore_none:
            return list(compress(self.filenames, self.find_corners))
        else:
            return self.filenames
    
    def getImagesCalibrated(self):
        return self.find_corners
    
    def getImagesAndPoints(self):
        return self.getImagesCalibrated(), self.getObjPoints(), self.getImgPoints()

# Testing camera calibration

### Tools
The following functions ....

In [ ]:
def undistortImage(img, camera):
    '''
        return a undistorted image using the Intrisic parameters of our Camera object.
        
        @params img: image which will be undistorted using the camera intrisic parameters.
        @params camera: Object wich contains the intrisic and extrinsic parameters of the Camera
    '''
    w,h = img.shape[:2]
    newcameramatrix, roi = cv2.getOptimalNewCameraMatrix(camera.K, camera.dist_coeff, (w,h), 1, (w,h))
    return cv2.undistort(img, camera.K, camera.dist_coeff, None, newcameramatrix)

def showUndistortImage(img, camera):
    '''
        Using matplotlib, show the undistorted images
        
        @params img: image which will be undistorted using the camera intrisic parameters.
        @params camera: Object wich contains the intrisic and extrinsic parameters of the Camera
    '''
    imgs = [img, undistortImage(img, camera)]
    
    fig = plt.figure(figsize=(20, 20))
    for imgIdx in range(len(imgs)):
        ax = plt.subplot(1,2,imgIdx+1)
        plt.xticks([])
        plt.yticks([])   
        ax.imshow(imgs[imgIdx], cmap='gray')
    
    plt.show()

### Synthetic test (Using OpenCV dataset)

It is necessary to create our `Camera` which will contain the **extrinsic** and **intrisic parameters**. Anyway, in order to calculate that parameters, it is necessary to use a `CameraCalibrator` which is necessary to pass the `Camera` that will be processed and the size of the `chessboard` that was used to calibrate our `Camera`. In this example, using OpenCV test, the `chessboard size` corresponds to (6,7).

In [ ]:
myCamera = Camera("MyCameraName")
myCalibrator = CameraCalibrator(myCamera, board_size=(6,7))

The `CameraCalibrator` contains a function called `calibrate()` which is necessary to pass the directory where are located our images captured by the `Camera`. The `show_calibration` parameter allows you to see the steps where the points on the chessboard are calculated.

**Important:** Some times OpenCV has several problems with GTK and `show_calibration` parameter is not able to use. This problem is presented on Linux.

In [ ]:
myCalibrator.calibrate("../images/Calibration/", show_calibration=True)

#### Visualization

Random image per execution...

In [ ]:
images = myCalibrator.getImagesFilenames()
img_idx = random.randint(0, len(images)-1)
img = cv2.imread(images[img_idx], 0)

showUndistortImage(img, myCamera)

#### Save calibration

The `Camera` parameters can be saved in a external file. For this purpose, the `saveCameraConfig` was implemented and the only parameter is the path where the file will be saved. The filename has the following structure: `[Camera name].yaml`.

In [ ]:
myCamera.saveCameraConfig(".")

#### Load a previous calibration

Parameter: `[Camera name].yaml` location...

In [ ]:
myCamera.loadCameraConfig("./MyCameraName.yaml")